In [ ]:
good = [0, 1, 3, 4, 8, 9, 11, 14, 23, 25, 27, 28, 30, 34, 36]
listed_bad = [2, 5, 7, 10, 13, 15, 16, 17, 18, 19, 20, 21, 22, 29, 33, 37, 52, 53]
confirmed_bad = [2, 7, 10, 12, 13, 15, 16, 17, 20, 22, 38, 52, 53]
neverseen_bad = [5, 18, 19, 21, 29, 33]
listed_blank = [6, 17, 18, 19, 24, 26, 32, 35, 39-51, 54, 55, 56, 59]
after_hours = [12]
odd_lot = [37]
neutral = [41]

In [30]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pandas_bokeh

import polygon_backfill as pb
import filters as flt
import market_cluster as mc

result_path="/Users/bobcolner/QuantClarity/data"

# .plot_bokeh(kind='hist', bins=50, sizing_mode="scale_height", disable_scientific_axes=True)
# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG' are other good diverging colormaps
# pd.set_option('precision', 2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# get data from polygon API 

trades = pb.get_ticks_date('GLD', '2020-08-03', 'trades')
df = pb.trades_to_df(trades)

In [ ]:
# get data from s3

# list_s3_symbol('SPY')
df = pb.get_s3_df('GLD', '2020-08-27')
# df = pb.get_s3_df(symbol='SPY', date='2020-08-28', columns=['exchange_epoch', 'price', 'size', 'condition'])

# df = pd.read_feather('data.feather', columns=['exchange_epoch', 'price', 'size', 'condition'])
# df['date_time'] = pd.to_datetime(df.exchange_epoch)

In [24]:
# get local market daily data

mdf = pd.read_parquet('data/mdf.parquet')

sdf = mdf[mdf.symbol=='SPY']

In [27]:
### apply jma filtering

jma = flt.appy_jma_filter(sdf.vwap.values, length=8, phase=60, power=2)

sdf['jma'] = jma
sdf[['vwap','jma']].plot_bokeh(sizing_mode="scale_height")

/Users/bobcolner/opt/miniconda3/envs/backfill/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Figure(id='1332', ...)

In [31]:
### market clustering

# %time df = read_market_daily(result_path)
%time mdf = pd.read_parquet('data/mdf.parquet')

%time npdf, sharpe_ratios = mc.normalize_market_df(mdf)

# %time npdf_resid = colwise_linreg(npdf)
%time npdf_resid = pd.read_parquet('data/npdf_resid.parquet')

# %time par_cor_mat = npdf_resid.corr()
%time resid_par_cor_mat = pd.read_parquet('data/partial_cor_mat.parquet')

%time details_df = pd.read_parquet('data/details_df.parquet')

# cluster_lables = cluster_sim_matrix(similarity=resid_par_cor_mat.abs())
%time cluster_lables = pd.read_parquet('data/cluster_lables.parquet')

# symbol_meta = join_symbol_data(details_df, cluster_lables, sharpe_ratios, mdf)
%time symbol_meta = pd.read_parquet('data/symbol_meta.parquet')

# cluster_coheasion = mc.get_cluster_coheasion(sim_df=resid_par_cor_mat.abs(), symbol_meta=symbol_meta, cluster_col='cluster_n200')

CPU times: user 821 ms, sys: 345 ms, total: 1.17 s
Wall time: 1.68 s
CPU times: user 1.86 s, sys: 445 ms, total: 2.3 s
Wall time: 3.24 s
CPU times: user 967 ms, sys: 201 ms, total: 1.17 s
Wall time: 1.31 s
CPU times: user 1.28 s, sys: 635 ms, total: 1.92 s
Wall time: 2.05 s
CPU times: user 44.6 ms, sys: 13.1 ms, total: 57.6 ms
Wall time: 64.8 ms
CPU times: user 5.34 ms, sys: 1.5 ms, total: 6.85 ms
Wall time: 7.13 ms
CPU times: user 18 ms, sys: 4.32 ms, total: 22.3 ms
Wall time: 24.2 ms


In [ ]:
mc.get

In [ ]:
symbol_meta['liquidy_rank'] = symbol_meta.groupby('cluster_n200')['daily_avg_dollar_volume'].rank(method='first')

symbol_meta = symbol_meta.sort_values(['cluster_n200', 'liquidy_rank']).reset_index()

In [ ]:
by_cluster = symbol_meta.groupby('cluster_n200')
result = []
for cluster, frame in by_cluster:
    counts = dict(frame['sector'].value_counts())
    counts.pop('', None)  # drop symbols w/o sector/industry info
    result.append(counts)

cluster_sec = pd.DataFrame(result)    
cluster_sec = cluster_sec.drop(columns=['Finance'])
cluster_sec_pct = cluster_sec.apply(lambda x: 100 * x / float(x.sum()), axis=1)

In [ ]:
cluster_sec_pct.style.background_gradient(cmap='coolwarm', axis=0)

In [ ]:
# n += 1
n = 61
print(n)
symbol_meta[symbol_meta.cluster_n200==n].sector.value_counts()

In [ ]:
sym_clust = symbol_meta[symbol_meta.cluster_n200==n]
sym_clust